In [1]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

2023-06-21 01:57:38.842381: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/jhohannes/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.constraints import maxnorm
from tensorflow.keras.utils import to_categorical


/home/jhohannes/miniconda3/envs/tf/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
dataset_dir = './_data'

train_ratio = 0.8  # 80% for training, 20% for testing
# Lists to store the file paths and labels
file_paths = []
label_ids = []
labels = ['A', 'B', 'C', 'D', 'E', 'Exceptions']

In [ ]:
import tensorflow as tf

devices = tf.config.list_physical_devices("GPU")

if devices:
  print("TensorFlow is running with your GPU")
else:
  print("TensorFlow is not running with your GPU")

## Checking if any image is corrupted or not

In [ ]:
import os
# Iterate through each class directory
for label_id, label in enumerate(labels):
    class_dir = os.path.join(dataset_dir, label)
    if os.path.isdir(class_dir):
        image_files = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        file_paths.extend([os.path.join(class_dir, f) for f in image_files])
        label_ids.extend([label_id] * len(image_files))

# Shuffle the file paths and label IDs
tf.random.set_seed(42)
tf.random.shuffle(file_paths)
tf.random.set_seed(42)
tf.random.shuffle(label_ids)

# Split the dataset into training and testing sets using stratified sampling
X_train_paths, X_test_paths, y_train_ids, y_test_ids = train_test_split(file_paths, label_ids, train_size=train_ratio, stratify=label_ids)

# Convert the image data into NumPy arrays
X_train = np.array([img_to_array(load_img(path, target_size=(130, 20))) for path in X_train_paths])
X_test = np.array([img_to_array(load_img(path, target_size=(130, 20))) for path in X_test_paths])

# Normalizing and performing one-hot encoding
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0

# Convert the label IDs into one-hot encoded vectors
y_train = to_categorical(y_train_ids, num_classes=len(labels))
y_test = to_categorical(y_test_ids, num_classes=len(labels))

num_classes = y_test.shape[1]
num_classes

In [ ]:
X_train.shape

In [ ]:
'./_data/Exceptions/Question2-scan0047.jpg' in X_train_paths

In [ ]:
import keras

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(130,20,3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))



In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

# create a learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 ** (epoch / 15))

In [ ]:
hist = model.fit(X_train, y_train , epochs=15, validation_split=0.1, callbacks=[lr_scheduler])

## Save Model

In [ ]:
models_path = "./saved_models"

os.makedirs(models_path, exist_ok=True)

# get the number of files in the directory
def get_num_files(path):
    if not os.path.exists(path):
        return 0
    return sum([len(files) for r, d, files in os.walk(path)])

# save the model with the current number
num_files = get_num_files(models_path)

model.save(os.path.join(models_path, f"model_{num_files}.h5"))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
plt.plot(hist.history['accuracy'], color='red', label='acc')
plt.plot(hist.history['val_accuracy'], label='val_acc')
fig.show()

In [ ]:
fig
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')

In [ ]:
predictions = model.predict(X_test)
rounded_predictions = np.apply_along_axis(lambda x: np.round(x), 1, predictions)

In [ ]:
predictions = tf.argmax(predictions, axis=1)


In [ ]:
y_test_argmax = tf.argmax(y_test, axis=1)

In [ ]:
cm = tf.math.confusion_matrix(y_test_argmax, predictions)
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

In [ ]:
cm.numpy()

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 10))
sns.heatmap(cm, annot=True, fmt=".2f", linewidths=0.5, square=True, cmap='Blues_r', xticklabels=labels, yticklabels=labels)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')
plt.title("Confusion matrix")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Accuracy, Recall, BinaryAccuracy

labels = y_test

precision = Precision()
accuracy = Accuracy()
recall = Recall()

In [ ]:
# Evaluate the metrics
precision.update_state(labels, rounded_predictions)
accuracy.update_state(labels, rounded_predictions)
recall.update_state(labels, rounded_predictions)

# Print the metrics
print('Precision:', precision.result().numpy())
print('Accuracy:', accuracy.result().numpy())
print('Recall:', recall.result().numpy())

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
import cv2

a = cv2.imread(dataset_dir + '/Exceptions/Question2-scan0047 - Copy.jpg')
resize = tf.image.resize(a, (130, 20))

b = model.predict(np.expand_dims(resize/255, 0))
plt.imshow(a)
e = np.apply_along_axis(lambda x: np.round(x), 1, b)
e